In [13]:
import subprocess
from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Corre el simulador como subproceso
subprocess.Popen(["python", "Datos/Bizums.py"])

# Configuración de Spark
spark = SparkSession.builder.appName("BizumMax").getOrCreate()

# Definir la ruta de entrada (puedes modificarla según tu estructura de directorios)
input_directory = "Datos/recibidos"

# Definir el esquema explícito para los datos CSV
schema = StructType([
    StructField("nombre", StringType(), True),
    StructField("importe", FloatType(), True),
    StructField("concepto", StringType(), True)
])

# Leer los archivos CSV con el formato adecuado desde el directorio en tiempo real
df = spark.readStream.schema(schema).option("sep", ";").option("header", "false").csv(input_directory)

# Renombramos las columnas a nombre, importe, concepto
df = df.selectExpr("_c0 as nombre", "_c1 as importe", "_c2 as concepto")

# Convertimos la columna 'importe' a tipo float
df = df.withColumn("importe", df["importe"].cast("float"))

# Realizamos el cálculo del máximo Bizum por persona
max_bizums = df.groupBy("nombre").agg({"importe": "max"}).withColumnRenamed("max(importe)", "max_bizum")

# Mostrar el resultado en la consola
query = max_bizums.writeStream.outputMode("complete").format("console").start()

# Mantener el proceso de streaming en ejecución
query.awaitTermination()

Py4JJavaError: An error occurred while calling o78.csv.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Traver

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window
from pyspark.sql.functions import explode, split

# Crear la sesión de Spark
spark = SparkSession.builder.appName("Ventana fija IABD WordCount").getOrCreate()

# Leer el flujo de datos desde un socket
dfLineas = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", "9999") \
    .option('includeTimestamp', 'true') \
    .load()

# Transformar los datos: separar las palabras y agregar la columna de timestamp
dfPalabras = dfLineas.select(
    explode(split(dfLineas.value, ' ')).alias('palabra'),
    dfLineas.timestamp
)

# Aplicar la ventana fija de 3 minutos con deslizamiento de 2 minutos
windowedCounts = dfPalabras.groupBy(
    window(dfPalabras.timestamp, "3 minutes", "2 minutes"), dfPalabras.palabra
).count().orderBy('window')

# Escribir la salida en la consola en modo completo
palabrasQuery = windowedCounts.writeStream \
    .format("console") \
    .outputMode("complete") \
    .option('truncate', 'false') \
    .start()

# Esperar a que termine el streaming
palabrasQuery.awaitTermination()

Py4JJavaError: An error occurred while calling o184.start.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:324)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:294)
	at org.apache.hadoop.fs.RawLocalFileSystem.createOutputStreamWithMode(RawLocalFileSystem.java:439)
	at org.apache.hadoop.fs.RawLocalFileSystem.create(RawLocalFileSystem.java:428)
	at org.apache.hadoop.fs.RawLocalFileSystem.create(RawLocal